In [2]:
import os
import sys
initial_path = set(sys.path)
CARLA_EGG_PATH = "/home/rudy/Documents/PythonAPI/carla/dist/carla-0.9.11-py3.7-linux-x86_64.egg"
sys.path.append(CARLA_EGG_PATH)

# ADD 
try:
    sys.path.append('/home/rudy/Documents/carla-dataset-runner/PythonAPI/carla')
except IndexError:
    pass

new_paths = set(sys.path) - initial_path
for path in new_paths:
    print(f"Added: {path} to the Path")


Added: /home/rudy/Documents/PythonAPI/carla/dist/carla-0.9.11-py3.7-linux-x86_64.egg to the Path
Added: /home/rudy/Documents/carla-dataset-runner/PythonAPI/carla to the Path


In [3]:
import carla
import random
import time
from agents.navigation.behavior_agent import BehaviorAgent
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.global_route_planner_dao import GlobalRoutePlannerDAO
import xml.etree.ElementTree as ET
import math
import logging

In [5]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town02')
world = client.get_world()
map_ = world.get_map()
blueprint_library = world.get_blueprint_library()
vehicle_bp = random.choice(blueprint_library.filter('vehicle.bmw.grandtourer'))

In [6]:
import os

class RouteParser:
    
    def __init__(self, map_, path: str):
        self.path = path
        self.map = map_
        
    def parse_file(self):
        if not os.path.exists(self.path):
            raise ValueError("Provided path doesn't exist in file directory")
        
        root = ET.parse(self.path).getroot()
        parsed_routes = {}
        for route in root:
            parsed_routes[route.attrib['id']] = {'town': route.attrib['town'], 'waypoints': []}
            for wp in route:
                if wp.tag == 'waypoint':
                    parsed_routes[route.attrib['id']]['waypoints'].append(self.parse_locs(wp.attrib))
        return parsed_routes
        
    
    def parse_locs(self, wp_as_dict):
        rotation = carla.Rotation(pitch=float(wp_as_dict['pitch']), yaw=float(wp_as_dict['yaw']), roll=float(wp_as_dict['roll']))
        location = carla.Location(x=float(wp_as_dict['x']), y=float(wp_as_dict['y']), z=float(wp_as_dict['z']))
        return location
        

In [7]:
class NPCClass:
    def __init__(self):
        logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)
        self.vehicles_list = []
        self.walkers_list = []
        self.all_id = []
        self.client = carla.Client('127.0.0.1', 2000)
        self.client.set_timeout(20.0)

    def create_npcs(self, number_of_vehicles=150, number_of_walkers=70):
        world = self.client.get_world()
        blueprints = world.get_blueprint_library().filter('vehicle.*')
        blueprints = [x for x in blueprints if not x.id.endswith('etron')]  # This guy has a wonky movement [physics broken?]
        blueprintsWalkers = world.get_blueprint_library().filter('walker.pedestrian.*')

        spawn_points = world.get_map().get_spawn_points()
        number_of_spawn_points = len(spawn_points)

        if number_of_vehicles < number_of_spawn_points:
            random.shuffle(spawn_points)
        elif number_of_vehicles > number_of_spawn_points:
            msg = 'requested %d vehicles, but could only find %d spawn points'
            logging.warning(msg, number_of_vehicles, number_of_spawn_points)
            number_of_vehicles = number_of_spawn_points

        # @todo cannot import these directly.
        SpawnActor = carla.command.SpawnActor
        SetAutopilot = carla.command.SetAutopilot
        FutureActor = carla.command.FutureActor

        # --------------
        # Spawn vehicles
        # --------------
        batch = []
        for n, transform in enumerate(spawn_points):
            if n >= number_of_vehicles:
                break
            blueprint = random.choice(blueprints)
            # Taking out bicycles and motorcycles, since the semantic/bb labeling for that is mixed with pedestrian
            if int(blueprint.get_attribute('number_of_wheels')) > 2:
                if blueprint.has_attribute('color'):
                    color = random.choice(blueprint.get_attribute('color').recommended_values)
                    blueprint.set_attribute('color', color)
                if blueprint.has_attribute('driver_id'):
                    driver_id = random.choice(blueprint.get_attribute('driver_id').recommended_values)
                    blueprint.set_attribute('driver_id', driver_id)
                blueprint.set_attribute('role_name', 'autopilot')
                batch.append(SpawnActor(blueprint, transform).then(SetAutopilot(FutureActor, True)))

        for response in self.client.apply_batch_sync(batch):
            if response.error:
                logging.error(response.error)
            else:
                self.vehicles_list.append(response.actor_id)

        # -------------
        # Spawn Walkers
        # -------------
        # 1. take all the random locations to spawn
        spawn_points = []
        for i in range(number_of_walkers):
            spawn_point = carla.Transform()
            loc = world.get_random_location_from_navigation()
            if (loc != None):
                spawn_point.location = loc
                spawn_points.append(spawn_point)

        # 2. we spawn the walker object
        batch = []
        for spawn_point in spawn_points:
            walker_bp = random.choice(blueprintsWalkers)
            # set as not invencible
            if walker_bp.has_attribute('is_invincible'):
                walker_bp.set_attribute('is_invincible', 'false')
            batch.append(SpawnActor(walker_bp, spawn_point))
        results = self.client.apply_batch_sync(batch, True)
        for i in range(len(results)):
            if results[i].error:
                logging.error(results[i].error)
            else:
                self.walkers_list.append({"id": results[i].actor_id})

        # 3. we spawn the walker controller
        batch = []
        walker_controller_bp = world.get_blueprint_library().find('controller.ai.walker')
        for i in range(len(self.walkers_list)):
            batch.append(SpawnActor(walker_controller_bp, carla.Transform(), self.walkers_list[i]["id"]))
        results = self.client.apply_batch_sync(batch, True)
        for i in range(len(results)):
            if results[i].error:
                logging.error(results[i].error)
            else:
                self.walkers_list[i]["con"] = results[i].actor_id
        # 4. we put altogether the walkers and controllers id to get the objects from their id
        for i in range(len(self.walkers_list)):
            self.all_id.append(self.walkers_list[i]["con"])
            self.all_id.append(self.walkers_list[i]["id"])
        self.all_actors = world.get_actors(self.all_id)

        # wait for a tick to ensure client receives the last transform of the walkers we have just created
        world.tick()
        world.wait_for_tick()

        # 5. initialize each controller and set target to walk to (list is [controler, actor, controller, actor ...])
        for i in range(0, len(self.all_id), 2):
            # start walker
            self.all_actors[i].start()
            # set walk to random point
            self.all_actors[i].go_to_location(world.get_random_location_from_navigation())
            # random max speed
            self.all_actors[i].set_max_speed(1 + random.random()/2)    # max speed between 1 and 2 (default is 1.4 m/s)
        print('Spawned %d vehicles and %d walkers' % (len(self.vehicles_list), len(self.walkers_list)))
        return self.vehicles_list, self.walkers_list

    def remove_npcs(self):
        print('Destroying %d NPC vehicles' % len(self.vehicles_list))
        self.client.apply_batch([carla.command.DestroyActor(x) for x in self.vehicles_list])

        # stop walker controllers (list is [controler, actor, controller, actor ...])
        for i in range(0, len(self.all_id), 2):
            self.all_actors[i].stop()

        print('Destroying %d NPC walkers' % len(self.walkers_list))
        self.client.apply_batch([carla.command.DestroyActor(x) for x in self.all_id])


In [18]:
def compute_distance(loc1, loc2):
    dx = loc1.x - loc2.x
    dy = loc1.y - loc2.y
    return math.sqrt(dx*dx + dy*dy)


def interpolate_trajectory(world, waypoints_trajectory, hop_resolution=1.0):
    """
    Given some raw keypoints interpolate a full dense trajectory to be used by the user.
    returns the full interpolated route both in GPS coordinates and also in its original form.
    
    Args:
        - world: an reference to the CARLA world so we can use the planner
        - waypoints_trajectory: the current coarse trajectory
        - hop_resolution: is the resolution, how dense is the provided trajectory going to be made
    """
    
    dao = GlobalRoutePlannerDAO(world.get_map(), hop_resolution)
    grp = GlobalRoutePlanner(dao)
    grp.setup()
    # Obtain route plan
    route = []
    for i in range(len(waypoints_trajectory) - 1):   # Goes until the one before the last.

        waypoint = waypoints_trajectory[i]
        waypoint_next = waypoints_trajectory[i + 1]
        interpolated_trace = grp.trace_route(waypoint, waypoint_next)
        
        for wp_tuple in interpolated_trace:
            route.append((wp_tuple[0], wp_tuple[1]))

    return route



In [9]:
parser = RouteParser(map_, 'routes_training.xml')
routes = parser.parse_file()
route = routes['1']

In [33]:
route['waypoints']

### How to travel using a predefined route

In [11]:
# example of how to travel using a predefined route

# 0. Some general settings
hop_resolution = 1.0
vehicles = 20
walkers = 10


# 1. Load world
client.load_world(route['town'])
world = client.get_world()
map_ = world.get_map()
print(f"World {route['town']} loaded")

World Town01 loaded


In [12]:
# 2.1 Interpolate waypoints 
world_route = interpolate_trajectory(world, route['waypoints'], hop_resolution=hop_resolution)

# 2.2 Draw waypoints
for i, t in enumerate(world_route):
    world.debug.draw_string(t[0].transform.location, str(i), draw_shadow=False, color=carla.Color(r=255, g=0, b=0), life_time=30, persistent_lines=True)
    
# 2.3 Spawn some NPCs

NPC = NPCClass()
vehicles_list, _ = NPC.create_npcs(vehicles, walkers)

ERROR: Spawn failed because of collision at spawn position
ERROR: Spawn failed because of collision at spawn position
ERROR: Spawn failed because of collision at spawn position


Spawned 16 vehicles and 7 walkers


In [32]:

closest_points = []
start_wp = world_route[0][0]
for p in map_.get_spawn_points():
    distance = compute_distance(start_wp.transform.location, p.location) 
    if distance < 10:
        closest_points.append((p, distance))

sorted_closest_points = sorted(closest_points, key=lambda p: p[1])
closest_points = [p[0] for p in sorted_closest_points]

closest_spawnable_point = None
for p in closest_points:
    wp_p = map_.get_waypoint(p.location)
    if wp_p.lane_id == start_wp.lane_id:
        closest_spawnable_point = p

vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
ego_vehicle = world.spawn_actor(vehicle_bp, p) 

In [21]:
# 3.2 Setup behavior agent

ego_agent = BehaviorAgent(ego_vehicle, behavior='cautious')
ego_agent.set_predefined_route(world_route)

In [22]:
# 4. Run!

while True:
    ego_agent.update_information()
    control = ego_agent.run_step()
    ego_vehicle.apply_control(control)

deque index out of range


AttributeError: 'NoneType' object has no attribute 'is_junction'